In [3]:
import socket
import select
import json
import time
from subprocess import Popen


In [4]:
RECEIVER_FILE = "run_receiver.py"

In [5]:
# Helper methods

def get_open_udp_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

def curr_ts_ms():
    if not hasattr(curr_ts_ms, 'epoch'):
        curr_ts_ms.epoch = time.time()
    return int((time.time() - curr_ts_ms.epoch) * 1000)


In [6]:
READ_FLAGS = select.POLLIN | select.POLLPRI
WRITE_FLAGS = select.POLLOUT
ERR_FLAGS = select.POLLERR | select.POLLHUP | select.POLLNVAL
READ_ERR_FLAGS = READ_FLAGS | ERR_FLAGS
ALL_FLAGS = READ_FLAGS | WRITE_FLAGS | ERR_FLAGS

In [66]:


class Sender(object):
    def __init__(self, port: int) -> None:
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind(('0.0.0.0', port))
        self.poller = select.poll()
        self.poller.register(self.sock, ALL_FLAGS)
        self.poller.modify(self.sock, ALL_FLAGS)
        self.peer_addr = None
        
        # Congestion control variables
        self.seq_num = 0
        self.next_ack = 0
        self.cwnd = 30.0
        self.step_len_ms = 10

        # Variables for the reinforment learning
        self.delivered_time = 0
        self.delivered = 0
        self.sent_bytes = 0

        self.min_rtt = float('inf')
        self.delay_ewma = None
        self.send_rate_ewma = None
        self.delivery_rate_ewma = None

        self.step_start_ms = None
        self.running = True
        
        self.unacknowledged_packets = {}
        self.rtts = []
        
    def window_is_open(self):
        # Returns true if the congestion window is not full
        return self.seq_num - self.next_ack < self.cwnd

    def send(self):
        serialized_data = json.dumps({
            'seq_num': self.seq_num,
            'send_ts': curr_ts_ms(),
            'sent_bytes': self.sent_bytes,
            'delivered_time': self.delivered_time,
            'delivered': self.delivered
        })
        self.unacknowledged_packets[self.seq_num] = True
        self.seq_num += 1
        self.sent_bytes += len(serialized_data.encode())
        self.sock.sendto(serialized_data.encode(), self.peer_addr)
    
    def recv(self):
        serialized_ack, addr = self.sock.recvfrom(1600)
        print(serialized_ack)
        if serialized_ack.decode() == "Hello from receiver":
            # Throwing away extra handshakes
            return
        ack = json.loads(serialized_ack.decode())
        del self.unacknowledged_packets[ack['seq_num']]

        self.next_ack = max(self.next_ack, ack['seq_num'] + 1)
        curr_time_ms = curr_ts_ms()
        rtt = float(curr_time_ms - ack['send_ts'])
        self.rtts.append(rtt)

    def handshake(self):
        """Handshake with peer receiver. Must be called before run()."""

        while True:
            msg, addr = self.sock.recvfrom(1600)
            

            if msg.decode() == 'Hello from receiver' and self.peer_addr is None:
                self.peer_addr = addr
                print("Sending data to receiver")
                self.sock.sendto('Hello from sender'.encode(), self.peer_addr)
                print('[sender] Handshake success! '
                                 'Receiver\'s address is %s:%s\n' % addr)
                break
        self.sock.setblocking(0)

    def run(self):
        curr_flags = ALL_FLAGS
        TIMEOUT = 1000  # ms
        start_time = time.time()

        while time.time() - start_time < 10:
            events = self.poller.poll(TIMEOUT)
            if not events:
                self.send()
            for fd, flag in events:
                assert self.sock.fileno() == fd

                if flag & ERR_FLAGS:
                    sys.exit('Error occurred to the channel')

                if flag & READ_FLAGS:
                    self.recv()

                if flag & WRITE_FLAGS:
                    if self.window_is_open():
                        self.send()

        
def print_performance(sender):
    print("Num Packets Lost: %s" % len(sender.unacknowledged_packets))
    print("Percent Packets Lost: %f" % float(len(sender.unacknowledged_packets))/sender.seq_num)
    print("Total bytes sent: %d" % sender.sent_bytes)
    print("Average RTT: %f" % float(sum(sender.rtts))/len(sender.rtts))

In [50]:

mahimahi_settings = [
    "mm-delay 28 mm-loss uplink 0.0477 mm-link %s %s --uplink-queue=droptail --uplink-queue-args=packets=14" % ('traces/0.57mbps-poisson.trace', 'traces/0.57mbps-poisson.trace'),
    'mm-delay 88 mm-link %s %s --uplink-queue=droptail --uplink-queue-args=packets=130' % ('traces/2.64mbps-poisson.trace', 'traces/2.64mbps-poisson.trace'),
    'mm-delay 130 mm-link %s %s --uplink-queue=droptail --uplink-queue-args=packets=426' % ('traces/3.04mbps-poisson.trace', 'traces/3.04mbps-poisson.trace'),
    'mm-delay 27 mm-link %s %s --uplink-queue=droptail --uplink-queue-args=packets=173' % ('traces/100.42mbps.trace', 'traces/100.42mbps.trace'),
    'mm-delay 45 mm-link %s %s --uplink-queue=droptail --uplink-queue-args=packets=450' % ('traces/114.68mbps.trace', 'traces/114.68mbps.trace')
]
mahimahi_cmd = mahimahi_settings[3]

In [51]:
port = get_open_udp_port()
sender = Sender(port)
cmd = "%s -- sh -c 'python %s $MAHIMAHI_BASE %s'" % (mahimahi_cmd, RECEIVER_FILE, port)
print(cmd)
process = Popen(cmd, shell=True)

mm-delay 27 mm-link traces/100.42mbps.trace traces/100.42mbps.trace --uplink-queue=droptail --uplink-queue-args=packets=173 -- sh -c 'python run_receiver.py $MAHIMAHI_BASE 48893'


In [52]:
sender.handshake()

Sending data to receiver
[sender] Handshake success! Receiver's address is 100.64.0.8:42666



In [53]:
sender.run()

b'{"delivered": 0, "seq_num": 0, "sent_bytes": 0, "ack_bytes": 88, "send_ts": 9708910, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 1, "sent_bytes": 88, "ack_bytes": 89, "send_ts": 9708910, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 2, "sent_bytes": 177, "ack_bytes": 90, "send_ts": 9708912, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 3, "sent_bytes": 267, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4, "sent_bytes": 357, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 5, "sent_bytes": 447, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 6, "sent_bytes": 537, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 7, "sent_bytes": 627, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 8, "sent_bytes": 717, "ack_bytes": 90, "send_ts": 9708913, "delivered_time": 0}'
b'{"

b'{"delivered": 0, "seq_num": 104, "sent_bytes": 9547, "ack_bytes": 93, "send_ts": 9709121, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 105, "sent_bytes": 9640, "ack_bytes": 93, "send_ts": 9709122, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 106, "sent_bytes": 9733, "ack_bytes": 93, "send_ts": 9709122, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 107, "sent_bytes": 9826, "ack_bytes": 93, "send_ts": 9709122, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 108, "sent_bytes": 9919, "ack_bytes": 93, "send_ts": 9709122, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 109, "sent_bytes": 10012, "ack_bytes": 94, "send_ts": 9709129, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 110, "sent_bytes": 10106, "ack_bytes": 94, "send_ts": 9709130, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 111, "sent_bytes": 10200, "ack_bytes": 94, "send_ts": 9709130, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 112, "sent_bytes": 10294, "ack_bytes": 94, "send_ts": 9

b'{"delivered": 0, "seq_num": 199, "sent_bytes": 18472, "ack_bytes": 94, "send_ts": 9709329, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 200, "sent_bytes": 18566, "ack_bytes": 94, "send_ts": 9709330, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 201, "sent_bytes": 18660, "ack_bytes": 94, "send_ts": 9709330, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 202, "sent_bytes": 18754, "ack_bytes": 94, "send_ts": 9709331, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 203, "sent_bytes": 18848, "ack_bytes": 94, "send_ts": 9709331, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 204, "sent_bytes": 18942, "ack_bytes": 94, "send_ts": 9709332, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 205, "sent_bytes": 19036, "ack_bytes": 94, "send_ts": 9709332, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 206, "sent_bytes": 19130, "ack_bytes": 94, "send_ts": 9709333, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 207, "sent_bytes": 19224, "ack_bytes": 94, "send_t

b'{"delivered": 0, "seq_num": 300, "sent_bytes": 27966, "ack_bytes": 94, "send_ts": 9709536, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 301, "sent_bytes": 28060, "ack_bytes": 94, "send_ts": 9709537, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 302, "sent_bytes": 28154, "ack_bytes": 94, "send_ts": 9709544, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 303, "sent_bytes": 28248, "ack_bytes": 94, "send_ts": 9709545, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 304, "sent_bytes": 28342, "ack_bytes": 94, "send_ts": 9709546, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 305, "sent_bytes": 28436, "ack_bytes": 94, "send_ts": 9709547, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 306, "sent_bytes": 28530, "ack_bytes": 94, "send_ts": 9709548, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 307, "sent_bytes": 28624, "ack_bytes": 94, "send_ts": 9709549, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 308, "sent_bytes": 28718, "ack_bytes": 94, "send_t

b'{"delivered": 0, "seq_num": 394, "sent_bytes": 36802, "ack_bytes": 94, "send_ts": 9709748, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 395, "sent_bytes": 36896, "ack_bytes": 94, "send_ts": 9709748, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 396, "sent_bytes": 36990, "ack_bytes": 94, "send_ts": 9709748, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 397, "sent_bytes": 37084, "ack_bytes": 94, "send_ts": 9709751, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 398, "sent_bytes": 37178, "ack_bytes": 94, "send_ts": 9709752, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 399, "sent_bytes": 37272, "ack_bytes": 94, "send_ts": 9709752, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 400, "sent_bytes": 37366, "ack_bytes": 94, "send_ts": 9709753, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 401, "sent_bytes": 37460, "ack_bytes": 94, "send_ts": 9709753, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 402, "sent_bytes": 37554, "ack_bytes": 94, "send_t

...

b'{"delivered": 0, "seq_num": 4146, "sent_bytes": 395716, "ack_bytes": 96, "send_ts": 9717866, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4147, "sent_bytes": 395812, "ack_bytes": 96, "send_ts": 9717866, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4148, "sent_bytes": 395908, "ack_bytes": 96, "send_ts": 9717867, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4149, "sent_bytes": 396004, "ack_bytes": 96, "send_ts": 9717867, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4150, "sent_bytes": 396100, "ack_bytes": 96, "send_ts": 9717867, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4151, "sent_bytes": 396196, "ack_bytes": 96, "send_ts": 9717867, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4152, "sent_bytes": 396292, "ack_bytes": 96, "send_ts": 9717879, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4153, "sent_bytes": 396388, "ack_bytes": 96, "send_ts": 9717879, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4154, "sent_bytes": 396484, "ack_b

b'{"delivered": 0, "seq_num": 4246, "sent_bytes": 405316, "ack_bytes": 96, "send_ts": 9718066, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4247, "sent_bytes": 405412, "ack_bytes": 96, "send_ts": 9718067, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4248, "sent_bytes": 405508, "ack_bytes": 96, "send_ts": 9718067, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4249, "sent_bytes": 405604, "ack_bytes": 96, "send_ts": 9718067, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4250, "sent_bytes": 405700, "ack_bytes": 96, "send_ts": 9718067, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4251, "sent_bytes": 405796, "ack_bytes": 96, "send_ts": 9718071, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4252, "sent_bytes": 405892, "ack_bytes": 96, "send_ts": 9718071, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4253, "sent_bytes": 405988, "ack_bytes": 96, "send_ts": 9718071, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4254, "sent_bytes": 406084, "ack_b

b'{"delivered": 0, "seq_num": 4354, "sent_bytes": 415684, "ack_bytes": 96, "send_ts": 9718273, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4355, "sent_bytes": 415780, "ack_bytes": 96, "send_ts": 9718276, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4356, "sent_bytes": 415876, "ack_bytes": 96, "send_ts": 9718277, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4357, "sent_bytes": 415972, "ack_bytes": 96, "send_ts": 9718279, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4358, "sent_bytes": 416068, "ack_bytes": 96, "send_ts": 9718279, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4359, "sent_bytes": 416164, "ack_bytes": 96, "send_ts": 9718279, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4360, "sent_bytes": 416260, "ack_bytes": 96, "send_ts": 9718280, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4361, "sent_bytes": 416356, "ack_bytes": 96, "send_ts": 9718280, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4362, "sent_bytes": 416452, "ack_b

b'{"delivered": 0, "seq_num": 4453, "sent_bytes": 425188, "ack_bytes": 96, "send_ts": 9718482, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4454, "sent_bytes": 425284, "ack_bytes": 96, "send_ts": 9718485, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4455, "sent_bytes": 425380, "ack_bytes": 96, "send_ts": 9718487, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4456, "sent_bytes": 425476, "ack_bytes": 96, "send_ts": 9718488, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4457, "sent_bytes": 425572, "ack_bytes": 96, "send_ts": 9718489, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4458, "sent_bytes": 425668, "ack_bytes": 96, "send_ts": 9718492, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4459, "sent_bytes": 425764, "ack_bytes": 96, "send_ts": 9718496, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4460, "sent_bytes": 425860, "ack_bytes": 96, "send_ts": 9718496, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4461, "sent_bytes": 425956, "ack_b

b'{"delivered": 0, "seq_num": 4557, "sent_bytes": 435172, "ack_bytes": 96, "send_ts": 9718695, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4558, "sent_bytes": 435268, "ack_bytes": 96, "send_ts": 9718695, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4559, "sent_bytes": 435364, "ack_bytes": 96, "send_ts": 9718695, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4560, "sent_bytes": 435460, "ack_bytes": 96, "send_ts": 9718704, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4561, "sent_bytes": 435556, "ack_bytes": 96, "send_ts": 9718705, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4562, "sent_bytes": 435652, "ack_bytes": 96, "send_ts": 9718705, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4563, "sent_bytes": 435748, "ack_bytes": 96, "send_ts": 9718705, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4564, "sent_bytes": 435844, "ack_bytes": 96, "send_ts": 9718705, "delivered_time": 0}'
b'{"delivered": 0, "seq_num": 4565, "sent_bytes": 435940, "ack_b

In [12]:
process.kill()

In [28]:
# Compute:
#   - the total number of bytes sent
#   - the average round trip time for a packet
#   - % of packets lost

In [32]:
curr_time = time.time()

In [33]:
time.time() - curr_time

9.630413055419922

In [39]:
avg([1,2,3])

NameError: name 'avg' is not defined

In [69]:
#print_performance(sender)
float(len(sender.unacknowledged_packets))/sender.seq_num

0.0064336264207591675